In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time 
from dateutil.parser import parse #to proccess dates
import re # to parse partial tags
import numpy as np

In [8]:
userAgent={'User-agent':'Mozilla/5.0'}
base_url='https://www.metacritic.com'
for ind in range(194):
    game_names=[]  #name
    scores=[]  #game score - what we will try and predict
    developers=[]  #developer name
    publishers=[]  #publisher name
    genres=[]  #genres the game is associated with
    ratings=[]  #game age rating
    user_score=[]  #game user score as aggragated by user reviews on site
    num_crit_reviews=[]  #number of critics reviews for game
    release_month=[]  #the month the game was released in
    release_year=[]  #the year the game was released in
    platforms=[]  #platform the game was reviewed for
    descriptions=[]  #game description for text proccessing
    dev_career_score=[]  #developer aggragated score
    pub_career_score=[]  #publisher aggragated score
    num_pos_critic=[]  #number of positive reviews
    num_mix_critic=[]  #number of mixed reviews
    num_neg_critic=[]  #number of negative reviews
    
    print('***************')
    # parses list page
    url_page=base_url+'/browse/games/score/metascore/all/all/filtered?page='+str(ind)
    print(url_page)
    response=requests.get(url_page,headers=userAgent)
    time.sleep(0.1) #we had problems with parsing the page so we added the pause so the page had time to load
    soup_page = BeautifulSoup(response.content, 'html.parser')
    # creates a list of all games in the page
    url_game_list=soup_page.find_all('a',attrs={'class':'title'})
    #**************#
    
    # parses each page
    for game in url_game_list:
        game_url=base_url+game['href']
        print(game_url)
        response = requests.get(game_url,headers=userAgent)
        time.sleep(0.4) #we had problems with parsing the page so we added the pause so the page had time to load
        soup_game = BeautifulSoup(response.content, 'html.parser')
        #**************#
    
        #game name
        game_names.append(soup_game.find("a",{"class":"hover_none"}).get_text().strip())
        #**************#

        #game score
        scores.append(int(soup_game.find("span",{"itemprop":"ratingValue"}).get_text().strip()))
        #**************#

        #list of genres
        genres_list=[]#the list of stripped down genres
        #finding the right tag
        genre_li=soup_game.find_all("li",attrs={"class":"summary_detail product_genre"})
        if(genre_li!=None):
            #receiving a list of all the genres
            gen_list=genre_li[0].find_all("span",attrs={"class":"data"})
            for genre in gen_list:
                genres_list.append(genre.get_text().strip()) #stripping names and adding them to list
            genres.append(list(dict.fromkeys(genres_list))) #removes duplicates inside genre list
        else:
            genres.append(np.nan)
        #**************#

        #rating
        #checking if rating exists, not all game get ratings - usually hyper violent games
        if(soup_game.find("li",attrs={"class":"summary_detail product_rating"})!=None):
            rating=soup_game.find("li",attrs={"class":"summary_detail product_rating"})
            ratings.append(rating.find("span",attrs={"class":"data"}).get_text().strip())
        else:
            ratings.append(np.nan)
        #**************#

        #user score
        user_score_element = soup_game.find("div",{"class":re.compile(r'^metascore_w user large game')})
        # some games didnt have an aggragated user score
        if(user_score_element == None or user_score_element.get_text().strip() == 'tbd'):
            user_score.append(np.nan)
        else:
            user_score.append(float(soup_game.find("div",{"class":re.compile(r'^metascore_w user large game')}).get_text().strip()))
        
        #**************#

        #number of reviews
        #reviews divided by positive,mixed and negative
        game_scores_div=soup_game("div",attrs={"class":"module reviews_module critic_reviews_module"})[0]
        score_list=game_scores_div.find_all("span",attrs={"class":"count"})
        sum_crit=0
        num_pos_critic.append(int(score_list[0].get_text().strip()))
        num_mix_critic.append(int(score_list[1].get_text().strip()))
        num_neg_critic.append(int(score_list[2].get_text().strip()))
        for spans in score_list:
            sum_crit=int(spans.get_text().strip())+sum_crit
        num_crit_reviews.append(sum_crit)
        #**************#

        #release date divide by day, month, year
        releas=soup_game("li",attrs={"class":"summary_detail release_data"})[0]
        date=releas.find("span",attrs={"class":"data"}).get_text().strip()
        dt=parse(date) #a function able to return a date from a string
        dt=dt.strftime('%Y-%m-%d') #a function able to return a date by format
        dt=dt.split(sep='-') #seperating the date
        release_month.append(int(dt[1]))
        release_year.append(int(dt[0]))
        #**************#

        #platforms
        platforms.append(soup_game.find("span",{"class":"platform"}).get_text().strip())
        #**************#

         #game description
        if(soup_game.find("li",{"class":"summary_detail product_summary"}) != None):
            desc=soup_game("li",attrs={"class":"summary_detail product_summary"})[0]
            #in some pages the summary is too long so they separate it to collapsed and expanded
            if(desc.find("span",attrs={"class":"blurb blurb_expanded"})==None):
                descr=desc.find("span",attrs={"class":"data"}).get_text().strip()
            else:
                descr=desc.find("span",attrs={"class":"blurb blurb_expanded"}).get_text().strip()
            descriptions.append(descr)
        else:
            descriptions.append(np.nan)
        #**************#

        #publisher
        #publisher career score
        pub_li=soup_game("li",attrs={"class":"summary_detail publisher"}) #finding the right tag
        #finding first publisher
        publisher_temp=pub_li[0].find("a").get_text().strip()
        publishers.append(publisher_temp)
        
        # finding the publisher url
        pub_url=pub_li[0].find("a")['href']
        pub_domain=base_url+pub_url
        response=requests.get(pub_domain,headers=userAgent)
        soup_pub = BeautifulSoup(response.content, 'html.parser')
        # appending the publisher score
        pub_career_temp=soup_pub.find("span",{"class":re.compile(r'^data textscore textscore_')}).get_text().strip()
        # some publisher didnt have a score, so we had to check
        if(pub_career_temp == None or pub_career_temp == 'N/A' or pub_career_temp == 'tbd'):
            pub_career_score.append(np.nan)
        else:
            pub_career_score.append(int(pub_career_temp))
        #**************#

        #developer
        #developer career score
        dev_li=soup_game("li",attrs={"class":"summary_detail developer"}) #finding the right tag
        #some games didnt have a developer written
        if(dev_li != None):
            developer_temp=dev_li[0].find("a").get_text().strip()        
            developers.append(developer_temp)
        else:
            developer_temp = publisher_temp
            developers.append(developer_temp)
        # appending the publisher score
        #checking if the publisher and developer aren't the same to save time
        if(developer_temp!=publisher_temp):
            dev_url=dev_li[0].find("a")['href']
            dev_domain=base_url+dev_url
            response=requests.get(dev_domain,headers=userAgent)
            soup_dev = BeautifulSoup(response.content, 'html.parser')
            dev_career_temp=soup_dev.find("span",{"class":re.compile(r'^data textscore textscore_')}).get_text().strip()
            if(dev_career_temp == None or dev_career_temp == 'N/A' or dev_career_temp == 'tbd'):
                dev_career_score.append(np.nan)
            else:
                dev_career_score.append(int(dev_career_temp))
        else:
            dev_career_score.append(int(pub_career_temp))
        #**************#
        #end of loop
        
    df=pd.DataFrame({'game_name': game_names, 'score': scores,'user_score': user_score,'platform': platforms,
                         'developer': developers,"developer_score":dev_career_score, 'publisher': publishers,
                         "publisher_score":pub_career_score,'rating': ratings,'release_month': release_month,
                         'release_year': release_year,'num_crit_review': num_crit_reviews, 'num_pos_critic': num_pos_critic,
                         'num_mix_critic': num_mix_critic,'num_neg_critic': num_neg_critic, 'genres':genres,
                         'descriptions': descriptions})
    if(ind>0): # on first time create the file
        df.to_csv("game_score_database.csv",mode='a',index=False,header=False)
    else: #on succesive runs append to existing file
        df.to_csv("game_score_database.csv",index=False)
    time.sleep(30)

***************
https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page=0
https://www.metacritic.com/game/nintendo-64/the-legend-of-zelda-ocarina-of-time
https://www.metacritic.com/game/playstation/tony-hawks-pro-skater-2
https://www.metacritic.com/game/playstation-3/grand-theft-auto-iv
https://www.metacritic.com/game/dreamcast/soulcalibur
https://www.metacritic.com/game/xbox-360/grand-theft-auto-iv
https://www.metacritic.com/game/wii/super-mario-galaxy
https://www.metacritic.com/game/wii/super-mario-galaxy-2
https://www.metacritic.com/game/xbox-one/red-dead-redemption-2
https://www.metacritic.com/game/xbox-one/grand-theft-auto-v
https://www.metacritic.com/game/playstation-3/grand-theft-auto-v
https://www.metacritic.com/game/pc/disco-elysium-the-final-cut
https://www.metacritic.com/game/xbox-360/grand-theft-auto-v
https://www.metacritic.com/game/dreamcast/tony-hawks-pro-skater-2
https://www.metacritic.com/game/switch/the-legend-of-zelda-breath-of-the-wild
https://

https://www.metacritic.com/game/switch/hades
https://www.metacritic.com/game/xbox/tony-hawks-pro-skater-3
https://www.metacritic.com/game/pc/half-life-alyx
https://www.metacritic.com/game/switch/divinity-original-sin-ii---definitive-edition
https://www.metacritic.com/game/pc/divinity-original-sin-ii
https://www.metacritic.com/game/pc/unreal-tournament-2004
https://www.metacritic.com/game/xbox-360/bioshock-infinite
https://www.metacritic.com/game/switch/ori-and-the-will-of-the-wisps
https://www.metacritic.com/game/xbox-360/braid
https://www.metacritic.com/game/playstation-2/god-of-war-ii
https://www.metacritic.com/game/wii-u/super-mario-3d-world
https://www.metacritic.com/game/pc/starcraft-ii-wings-of-liberty
https://www.metacritic.com/game/playstation-2/ssx
https://www.metacritic.com/game/xbox-360/street-fighter-iv
https://www.metacritic.com/game/pc/minecraft
https://www.metacritic.com/game/switch/undertale
https://www.metacritic.com/game/playstation-vita/persona-4-golden
https://www.m

In [9]:
df

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,genres,descriptions
0,Jet Grind Radio,94,8.3,Dreamcast,Smilebit,85,Sega,72,T,10,2000,24,24,0,0,"[Action, Platformer, 3D]","Join a graffiti crew, stamp your territory and..."
1,Metal Gear Solid,94,9.1,PlayStation,KCEJ,71,Konami,67,M,10,1998,20,19,1,0,"[Action Adventure, Modern]","You are Snake, a government agent on a mission..."
2,Grim Fandango,94,9.1,PC,LucasArts,73,LucasArts,73,T,10,1998,15,15,0,0,"[Adventure, General, 3D, Third-Person, Fantasy]",Something's rotten in the Land of Dead and you...
3,Tom Clancy's Splinter Cell Chaos Theory,94,9.1,Xbox,Ubisoft Montreal,74,Ubisoft,69,M,3,2005,70,70,0,0,"[Action Adventure, Modern]","As Sam Fisher, Third Echelon's most skilled Sp..."
4,Resident Evil Code: Veronica,94,9.4,Dreamcast,NexTech,94,Capcom,74,M,2,2000,18,18,0,0,"[Action Adventure, Survival]",Resident Evil Code: Veronica takes up the stor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Divinity: Original Sin II - Definitive Edition,92,7.6,Xbox One,Larian Studios Games,83,Larian Studios Games,83,M,8,2018,12,12,0,0,"[Role-Playing, Western-Style]","Set a thousand years after the first game, Div..."
96,NFL 2K3,92,6.5,GameCube,Visual Concepts,79,Sega,72,E,8,2002,24,24,0,0,"[Sports, Traditional, Football, Sim]",NFL 2K3 upgrades Sega Sports' football series ...
97,SSX Tricky,92,8.9,PlayStation 2,EA Canada,75,EA Sports Big,77,E,11,2001,33,33,0,0,"[Sports, Alternative, Snowboarding]",Second-generation rider physics provide more i...
98,NHL 2002,92,8.3,PlayStation 2,EA Sports,78,Electronic Arts,73,E,9,2001,22,22,0,0,"[Sports, Traditional, Ice Hockey, Sim]",The PlayStation 2 version features include Car...


In [10]:
df2=pd.read_csv('game_score_database.csv')

In [14]:
df2.mode()

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,genres,descriptions
0,Grand Theft Auto V,93.0,8.8,PC,Nintendo,74.0,Nintendo,74.0,M,11.0,2003.0,24.0,24.0,0.0,0.0,"['Action', 'Shooter', 'First-Person', 'Sci-Fi'...",A New World Created By Hidetaka Miyazaki And G...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,Oblivion is a single-player game that takes pl...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The next chapter in the Elder Scrolls saga arr...


In [19]:
df2.describe()

,score,user_score,developer_score,publisher_score,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic
count,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,94.250000,8.390500,80.100000,75.35000,7.460000,2007.525000,48.455000,48.085000,0.365000,0.005000
std,1.539268,0.728777,6.425452,5.96451,3.308038,6.912182,29.028109,28.763247,0.827769,0.070711
min,92.000000,5.700000,57.000000,64.00000,1.000000,1996.000000,7.000000,7.000000,0.000000,0.000000
25%,93.000000,8.200000,76.000000,72.00000,4.000000,2002.000000,22.000000,22.000000,0.000000,0.000000
50%,94.000000,8.600000,80.000000,74.00000,9.000000,2007.000000,44.000000,44.000000,0.000000,0.000000
75%,95.000000,8.900000,84.000000,78.00000,10.000000,2012.000000,72.000000,71.250000,1.000000,0.000000
max,99.000000,9.400000,94.000000,94.00000,12.000000,2022.000000,121.000000,118.000000,8.000000,1.000000


In [20]:
df2.mode()

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,genres,descriptions
0,Grand Theft Auto V,93.0,8.8,PC,Nintendo,74.0,Nintendo,74.0,M,11.0,2003.0,24.0,24.0,0.0,0.0,"['Action', 'Shooter', 'First-Person', 'Sci-Fi'...",A New World Created By Hidetaka Miyazaki And G...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,Oblivion is a single-player game that takes pl...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The next chapter in the Elder Scrolls saga arr...


In [29]:
df2.isna()

,game_name,score,user_score,platform,developer,developer_score,publisher,publisher_score,rating,release_month,release_year,num_crit_review,num_pos_critic,num_mix_critic,num_neg_critic,genres,descriptions
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
196,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
197,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
198,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
